In [38]:
from __future__ import print_function    # (at top of module)
import warnings
#warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import config

In [39]:
# Read the data from the file
data = pd.read_csv('data_500_entries_youtube.csv')
print("Number of entries in original data: " + str(len(data.index)))
data.head()

Number of entries in original data: 570


,song_id,song_title,artist,popularity,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,youtube_video_title
0,spotify:track:5ygDXis42ncn6kYG14lEVG,Baby Shark,[Pinkfong],77,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1,1956582159,Baby Shark Dance | Sing and Dance! | Animal So...
1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Girls Like You (feat. Cardi B),"[Maroon 5, Cardi B]",86,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1,1300452389,Maroon 5 - Girls Like You ft. Cardi B
2,spotify:track:6De0lHrwBfPfrhorm9q1Xl,Me Rehúso,[Danny Ocean],83,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1,1229501096,Danny Ocean - Me Rehúso (Official Audio)
3,spotify:track:1j6xOGusnyXq3l6IryKF3G,Déjala Que Vuelva (feat. Manuel Turizo),"[Piso 21, Manuel Turizo]",74,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1,1216075058,Piso 21 - Déjala Que Vuelva (feat. Manuel Turi...
4,spotify:track:2ijef6ni2amuunRoKTlgww,Sin Pijama,"[Becky G, Natti Natasha]",90,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0,1071141995,Becky G Natti Natasha - Sin Pijama (Video Ofic...


In [40]:
if 'song_id' in data.columns:
    data = data.drop_duplicates(subset=['song_id'], keep='first')
else:
    data = data.drop_duplicates(subset=['song_title'], keep='first')
    
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 570


In [41]:
data = data[data.popularity > 50]
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 570


In [42]:
# Make a copy of the data to which we will ad labels and then remove any 
# columns that we will not need
# This is currently a duplicate of the functionality above - could maybe only do this in one place

final_data = data.copy()
threshold = 90
labels = []
labeled_popular = 0
labeled_notpopular = 0
for item in data['popularity']:
    if item > threshold:
        labels.append(1)
        labeled_popular = labeled_popular + 1
    else:
        labels.append(0)
        labeled_notpopular = labeled_notpopular + 1
final_data['is_popular'] = labels

print('Number of popular examples after thresholding : ', labeled_popular)
print('Number of not popular examples after thresholding : ', labeled_notpopular)

# Drop unnecessary columns from original data
if 'song_id' in data.columns:
    final_data.drop(['song_id', 'song_title', 'artist', 'popularity', 'youtube_video_title'], 1, inplace=True)
else:
    final_data.drop(['song_title', 'artist', 'popularity'], 1, inplace=True) 


Number of popular examples after thresholding :  47
Number of not popular examples after thresholding :  523


In [53]:
from sklearn.preprocessing import scale, MinMaxScaler

COLUMNS_TO_SCALE = ["energy", "liveness", "tempo", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "time_signature", "danceability", "key", 
                    "duration", "loudness", "valence", "mode", "youtube_view_count"]
# Keep data in a temp variable for testing
scaled_data = final_data.copy()

# Normalization
scaled_data[COLUMNS_TO_SCALE] = MinMaxScaler().fit_transform(scaled_data[COLUMNS_TO_SCALE])

#Standardization
#scaled_data[COLUMNS_TO_SCALE] = scale(scaled_data[COLUMNS_TO_SCALE])

# Copy data back
final_data = scaled_data.copy()

#Just to check that everything is fine
final_data.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,is_popular
0,0.838293,0.331118,0.360693,0.399286,0.247690,0.000000,0.75,0.789030,0.636364,0.016514,0.807462,0.522258,1.0,1.000000,0
1,0.502527,0.111965,0.431906,0.049366,0.574298,0.000000,0.75,0.825598,0.000000,0.442256,0.594084,0.444083,1.0,0.664655,0
2,0.797866,0.028251,0.287019,0.083466,0.023312,0.000000,0.75,0.675105,0.090909,0.351029,0.627563,0.420195,1.0,0.628392,0
3,0.779899,0.055152,0.756132,0.104877,0.048692,0.000000,0.75,0.586498,0.090909,0.395074,0.762286,0.868621,1.0,0.621530,0
4,0.731611,0.084961,0.209243,0.041237,0.357908,0.000032,0.75,0.741210,1.000000,0.298565,0.804504,0.847991,0.0,0.547456,0


In [44]:
# X will be our examples and y will be our labels
X = final_data.drop('is_popular', axis=1)
y = final_data['is_popular']
# Sanity checks
print("Number of entries in actual data: " + str(len(X.index)))
print("Number of entries in label data: " + str(len(y.index)))

Number of entries in actual data: 570
Number of entries in label data: 570


In [45]:
X = X.drop('youtube_view_count', axis=1)

In [46]:
X.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode
0,0.838293,0.331118,0.360693,0.399286,0.247690,0.000000,0.75,0.789030,0.636364,0.016514,0.807462,0.522258,1.0
1,0.502527,0.111965,0.431906,0.049366,0.574298,0.000000,0.75,0.825598,0.000000,0.442256,0.594084,0.444083,1.0
2,0.797866,0.028251,0.287019,0.083466,0.023312,0.000000,0.75,0.675105,0.090909,0.351029,0.627563,0.420195,1.0
3,0.779899,0.055152,0.756132,0.104877,0.048692,0.000000,0.75,0.586498,0.090909,0.395074,0.762286,0.868621,1.0
4,0.731611,0.084961,0.209243,0.041237,0.357908,0.000032,0.75,0.741210,1.000000,0.298565,0.804504,0.847991,0.0


# Exhaustive search

In [47]:
import itertools
columns = ["energy", "liveness", "tempo", "speechiness", "acousticness", "instrumentalness", "time_signature", "danceability",
          "key", "duration", "loudness", "valence", "mode"]
allSubsets = []
for m in range(1, len(columns) + 1, 1):
    allSubsets.append(list(itertools.combinations(columns, m)))
print(allSubsets)

[[('energy',), ('liveness',), ('tempo',), ('speechiness',), ('acousticness',), ('instrumentalness',), ('time_signature',), ('danceability',), ('key',), ('duration',), ('loudness',), ('valence',), ('mode',)], [('energy', 'liveness'), ('energy', 'tempo'), ('energy', 'speechiness'), ('energy', 'acousticness'), ('energy', 'instrumentalness'), ('energy', 'time_signature'), ('energy', 'danceability'), ('energy', 'key'), ('energy', 'duration'), ('energy', 'loudness'), ('energy', 'valence'), ('energy', 'mode'), ('liveness', 'tempo'), ('liveness', 'speechiness'), ('liveness', 'acousticness'), ('liveness', 'instrumentalness'), ('liveness', 'time_signature'), ('liveness', 'danceability'), ('liveness', 'key'), ('liveness', 'duration'), ('liveness', 'loudness'), ('liveness', 'valence'), ('liveness', 'mode'), ('tempo', 'speechiness'), ('tempo', 'acousticness'), ('tempo', 'instrumentalness'), ('tempo', 'time_signature'), ('tempo', 'danceability'), ('tempo', 'key'), ('tempo', 'duration'), ('tempo', 'l

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

steps_taken = 0
scores_list = []

# Loop thorugh all the different subsets of 1,2,3, ..., (#features) elements
for n_elem_subsets in range(0, len(columns), 1):
    len_elem_subs = len(allSubsets[n_elem_subsets])
    best_score = 0
    feature_set = []
    # Loop through all the subsets of n_elem_subsets elements
    for cur_comb in range(0, len_elem_subs, 1):
        
        # Get the data with the current subset of features
        cur_data = X[list(allSubsets[n_elem_subsets][cur_comb])]
        
        # Transform to np array for faster computation
        cur_data_np = np.array(cur_data)
        
        # Counter of number of interations
        steps_taken += 1
        
        # Instantiate model
        model = LogisticRegression(random_state=4)
        
        # Instantiate cross validation strategy
        cv = StratifiedKFold(n_splits=10, random_state=3)
        
        # Compute the cross valdation scores
        scores = cross_val_score(model, cur_data_np, y, cv=cv, scoring='roc_auc')
        
        mean_score = scores.mean()
        
        # For brevity, we only keep the best feature subset of one size
        # Alternatively, we could keep the scores for all the different combinations
        if mean_score > best_score:
            best_score = mean_score
            feature_set = list(allSubsets[n_elem_subsets][cur_comb])
    #Keep the best score in a list    
    scores_list.append([feature_set, best_score])

print("Number of iterations: ", steps_taken)

c:\python27\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Number of iterations:  8191


In [50]:
# Print feature sets with best scores
for cur_list in scores_list:
    print(cur_list)

[['valence'], 0.65410014513788106]
[['tempo', 'valence'], 0.65825290275761972]
[['time_signature', 'loudness', 'valence'], 0.6631386066763425]
[['tempo', 'acousticness', 'loudness', 'valence'], 0.66415820029027572]
[['energy', 'tempo', 'acousticness', 'loudness', 'valence'], 0.67060050798258353]
[['energy', 'tempo', 'acousticness', 'time_signature', 'loudness', 'valence'], 0.67004535558780831]
[['energy', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'loudness', 'valence'], 0.66908381712626996]
[['energy', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'duration', 'loudness', 'valence'], 0.66447568940493473]
[['energy', 'liveness', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'duration', 'loudness', 'valence'], 0.65781023222060964]
[['energy', 'liveness', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'duration', 'loudness', 'valence'], 0.64912010159651667]
[['energy', 'liveness', 'tempo', 'speechiness',

# Filter methods

In [52]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=5)

sel = SelectKBest(chi2, k=2)
sel.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function chi2 at 0x14DFECF0>)